# This notebook for running a centralized version of the source seeking algorithm

In [5]:

%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation, rc


import pickle as pkl
import networkx as nx
from matplotlib import style
from functools import partial
import sys

sys.path.append('../../')

from utils.dLdp import analytic_dLdp,analytic_dhdz,analytic_dhdq,analytic_FIM
from utils.CentralizedEKF import CentralizedEKF

import time

%load_ext autoreload
%autoreload 2

In [2]:
def single_meas_func(C1,C0,k,b,dist):
    return k*(dist-C1)**b+C0

def joint_meas_func(C1s,C0s,ks,bs,x,ps):

    # Casting for the compatibility of jax.numpy

    C1=np.array(C1s)
    C0=np.array(C0s)
    k=np.array(ks)
    b=np.array(bs)
    p=np.array(ps)

    # Keep in mind that x is a vector of [q,q'], thus only the first half of components are observable.    
    dists=np.linalg.norm(x[:len(x)//2]-p,axis=1)

    return single_meas_func(C1,C0,k,b,dists) 

In [3]:
def centralized_main(q,N_trails,N_sen,N_iter):
    '''Experiment Parameters'''
  
    C_gain=0.1

    # Set up virtual sensors
    C1=-0.3 # Setting C1 as a negative number mitigates the blowing-up effect when the sensors are close to the source.
    C0=0
    k=1
    b=-2
   
    noise_std = 0.01
    minimum_sensing_reading=1e-5

    # The step size of each sensor
    max_linear_speed=0.1

    # Terminal condition
    contact_radius = 0.1


    t=time.time()
    data={'p':[],'qhat':[],'q':[]}

    for _ in range(N_trails):

        '''Initialize Key Data Structures'''
        p_0 = np.random.rand(N_sen,2)
        # qhat_0 = np.array([3,3])
        
        qhat_0 = np.array([4,2])
        # qhat_0 = np.random.rand(2)
        
        p = np.array(p_0) # Sensor Positins
        qhat = np.array(qhat_0)

        # The Centralized EKF
        estimator = CentralizedEKF(qhat_0)

        C1s=C1*np.ones(N_sen)
        C0s = C0*np.ones(N_sen)
        ks = k * np.ones(N_sen)
        bs = b*np.ones(N_sen)

        h=partial(joint_meas_func,C1s,C0s,ks,bs)# Freeze the coefficients, the signature becomes h(z,ps))
        dhdz=partial(analytic_dhdz,C1s=C1s,C0s=C0s,ks=ks,bs=bs)
        dhdqs=partial(analytic_dhdq,C1s=C1s,C0s=C0s,ks=ks,bs=bs)

        '''Main Loop'''

        p_history = []
        qhat_history = []
        q_history = []
        for _ in range(N_iter):
#             if np.min(np.linalg.norm(p-q,axis=1))>contact_radius:#Move and estimate only when not touching the source.
            # Measure
            r = np.linalg.norm(q-p,axis=1)
            y = k* ((r-C1)**b)+C0 + np.random.randn(N_sen)*noise_std
            y[y<=0]=minimum_sensing_reading # We don't want y to be zero or negative.

            # Estimate

            qhat=estimator.update_and_estimate_loc(h,dhdz,y,p)

            # Gradient update

            dp=analytic_dLdp(qhat,p,C1,C0,k,b)
            p-=max_linear_speed*(dp.T/np.linalg.norm(dp,axis=1)).T

            # Record data
            p_history.append(np.array(p))
            qhat_history.append(np.array(qhat))  
            q_history.append(np.array(q))

            # Check terminal condition
            
        data['q'].append(q_history)
        data['p'].append(np.array(p_history))
        data['qhat'].append(np.array(qhat_history))

    print('Time:',time.time()-t)
    return data

In [6]:
from joblib import Parallel,delayed

def simulate(N_sen,N_trails,N_iter):
    
    SEED = 45
    np.random.seed(SEED)
    data={}
    data['centralized']= centralized_main(q,N_trails,N_sen,N_iter)
    filepath = "../../Data/Centralized-{}Sensor.pkl".format(N_sen)
    with open(filepath,'wb') as file:
        pkl.dump(data,file)



q = np.array([6,6])  
N_trails = 100
N_iter = 150


for N_sen in [4,10,20,40]:
    simulate(N_sen,N_trails,N_iter)

Time: 1.2862849235534668
Time: 2.6856930255889893
Time: 3.721872329711914
Time: 4.393543004989624
